In [3]:
# import necessary libraries
import pandas as pd
import os
import glob

import pickle

In [5]:
# Change directory one step back and save as the root directory
ROOT_DIR = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# Change direcotry to the 'data\raw' folder
path = ROOT_DIR + '\\data\\raw'
print(path)

D:\gitProjects\springboard_capstone_1\Springboard_Capstone_01\data\raw


In [6]:
def get_file_name(file_name):
    name_ext = file_name.split("\\")[-1]
    name = name_ext.split(".")[0]
    
    return name

In [7]:
# use glob to get all the csv files in the folder
csv_files = glob.glob(os.path.join(path, "*.csv"))

# Initialize list; this is the final list that will store all the data from the json pull. 
read_data = []

# loop over the list of csv files
for f in csv_files:
        
    # read the csv file    
    df = pd.read_csv(f, sep = ',', thousands=',')
    
    if ' Close' in df.columns:
        #rename column name
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close']
        value = 'Close'
        format_date = '%m/%d/%y'
        
        
    if 'Price' in df.columns:
        df.columns = ['Date', 'Price', 'Open','High','Low','Vol','Change']
        value = 'Price'
        format_date = '%m/%d/%Y'
    
    #select columns
    df = df[['Date', value]]

    #change data type to date and numeric
    df['value'] = pd.to_numeric(df[value])
    df['Date'] = pd.to_datetime(df['Date'], format=format_date, errors="raise")
    
    df = df.drop(columns = [value])
    df = df.rename(columns = {'value': get_file_name(f)}, errors="raise")
    
    #set Date as index
    df.set_index('Date', drop=True, inplace=True)
    
    print('Sucess:', get_file_name(f))
    #display(df.head())
    
    read_data.append(df)
    # print the success report
    


Sucess: COMEX_Gold
Sucess: DJIA
Sucess: LME_Copper
Sucess: NASDAQ_COMP
Sucess: SP500_SPX


In [8]:
for i in range(len(read_data)):
    name = read_data[i].columns[0]
    size = len(read_data[i])
    phrase = str(i) + ': '+ name + ' (1*' + str(size) + ')'
    print(phrase)

0: COMEX_Gold (1*274)
1: DJIA (1*5724)
2: LME_Copper (1*274)
3: NASDAQ_COMP (1*5725)
4: SP500_SPX (1*5724)


In [9]:
def monthly_average(df):
            
    df['month'] = pd.DatetimeIndex(df.index).month
    df['year'] = pd.DatetimeIndex(df.index).year
    
    dfg = df.groupby(['year','month'],as_index=False).mean()
    dfg['Date'] = pd.to_datetime(dict(year = dfg.year, month = dfg.month, day = 1))
    
    dfg = dfg.drop(columns = ['month','year'])
    
    dfg.set_index('Date', drop=True, inplace=True)
    
    return dfg

In [10]:
df = read_data[0]
dfg = monthly_average(df)
dfg.index = pd.to_datetime(df.index, format = '%Y%m',errors='coerce')

for df_temp in read_data[1:]:
    dfg_temp = monthly_average(df_temp)
    dfg_temp.index = pd.to_datetime(dfg_temp.index, format = '%Y%m%d',errors='coerce')
    dfg = pd.merge_ordered(dfg, dfg_temp, on = 'Date')

In [11]:
display(dfg)

,Date,COMEX_Gold,DJIA,LME_Copper,NASDAQ_COMP,SP500_SPX
0,2000-01-01,1671.55,11281.263000,0.8440,4013.494000,1425.585500
1,2000-02-01,1684.90,10541.929500,0.7870,4410.871000,1388.874000
2,2000-03-01,1726.20,10483.390435,0.8000,4802.988261,1442.213043
3,2000-04-01,1778.70,10944.310526,0.7900,3863.637368,1461.355263
4,2000-05-01,1807.30,10580.267727,0.8110,3528.416818,1418.479545
...,...,...,...,...,...,...
269,2022-06-01,274.80,31446.712857,3.7145,11418.919048,3898.946667
270,2022-07-01,274.70,31535.323500,3.4295,11622.631500,3911.729500
271,2022-08-01,281.40,33009.564783,3.5185,12570.260435,4158.563043
272,2022-09-01,294.20,30649.559524,3.4070,11413.214091,3850.520476


In [15]:
# Change direcotry to the 'data\raw' folder
save_dir = ROOT_DIR + '\\data\\interim\\'

# Set a file name
file_save = save_dir + 'offline_data_raw.csv'
dfg.to_csv(file_save)

print('Save:\n', file_save)

Save:
 D:\gitProjects\springboard_capstone_1\Springboard_Capstone_01\data\interim\offline_data_raw.csv
